如果要加入新資料
1. NewDataLoader(BasicLoader)
2. LoaderMapping新增欄位
3. .yml新增欄位

In [2]:
### reload magic
%reload_ext autoreload
%autoreload 2

import os
import sys
import yaml
import argparse
import torch
import torchvision
import torch.utils.cpp_extension

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from src.data_manager import DataManager
from src.model_builder import ModelBuilder

# os.environ['CUDA_VISIBLE_DEVICES']="0"
print('Python version', sys.version)
print('CUDA_HOME', torch.utils.cpp_extension.CUDA_HOME)
print('CudaToolKit Version', torch.version.cuda)
print('Cudnn Version', torch.backends.cudnn.version())
print('torch Version', torch.__version__)
print('torchvision Version', torchvision.__version__)

config_file = './exp_config/exp1.yml'
with open(config_file, "r") as content:
    config = yaml.safe_load(content)

NameError: name '_C' is not defined

In [2]:
dm = DataManager(
    data_meta_info=config['train_config']['data_meta_info'], 
    **config['train_config']['data_loader_params']
)

100%|██████████| 366595/366595 [00:02<00:00, 141368.68it/s]


RainLoader  instantiated.


100%|██████████| 366871/366871 [00:02<00:00, 150988.31it/s]


RadarLoader  instantiated.
[DataManager] Training Data Size: 214224; Validating Data Size: 71408; Testing Data Size: 71409


In [3]:
### the followings are lagecy code
model = ModelBuilder(
    **config['model'],
    data_info = dm.get_data_info()
).build()

Using BalancedGRUAdvPONI model
[Forecaster_PONI] TargetLen:3 TeacherForcing:0.5
[EncoderParams] channel_count:2 Shape:(120, 120)
[Discriminator] Downsample:3
[BalAdvPoniModel W:0.01] Ckp:/wk171/handsomedong/deepQPF_research/checkpoint/ 


In [ ]:
logger = TensorBoardLogger(save_dir='logs', name=config['model']['name'])
checkpoint_callback = model.get_checkpoint_callback()
trainer = Trainer(
    benchmark=True,
    accelerator='gpu',
    devices=[1],
    max_epochs=50, 
    fast_dev_run=False, 
    logger=logger,
    check_val_every_n_epoch=1,
    callbacks=[checkpoint_callback, EarlyStopping(monitor="val_loss", patience=5)],
)
trainer.fit(model, dm)